In [1]:
import numpy as np
import pymannkendall as mk

In [4]:
pre=np.array([12,24,43,53,130,102,140,159,133,84,42,26,29,46,37,66,71,103,150,143,129,75,33,12])

例一：计算sens斜率并进行原始Mann-Kendall检验

In [5]:
result = mk.original_test(pre,alpha=0.05)

#pre是降水数据，alpha是置信区间
#OUT变量对应上述
'''
Mann_Kendall_Test(
     trend='no trend',
     h=False,
     p=0.5851588651847224, 
     z=0.5458648159491473,
     Tau=0.08333333333333333,
     s=23.0, var_s=1624.3333333333333,
     slope=0.8500000000000001,
     intercept=58.725)'''

"\nMann_Kendall_Test(\n     trend='no trend',\n\xa0\xa0\xa0\xa0\xa0h=False,\n\xa0\xa0\xa0\xa0\xa0p=0.5851588651847224,\xa0\n     z=0.5458648159491473,\n     Tau=0.08333333333333333,\n     s=23.0, var_s=1624.3333333333333,\n     slope=0.8500000000000001,\n\xa0\xa0\xa0\xa0\xa0intercept=58.725)"

当只进行Sen's斜率计算时：

In [9]:
result = mk.sens_slope(pre)
print(result)

Sens_Slope_Test(slope=0.8500000000000001, intercept=58.725)


例二：计算季节Sen's斜率并进行季节MK检验

In [10]:
result = mk.seasonal_test(pre,alpha=0.05,period=12)
print(result)

Seasonal_Mann_Kendall_Test(trend='no trend', h=False, p=0.7728299926844475, z=-0.2886751345948129, Tau=-0.16666666666666666, s=-2.0, var_s=12.0, slope=-5.0, intercept=73.29166666666667)


当只进行季节Sen's斜率计算时：

In [12]:
result = mk.seasonal_sens_slope(pre,period=12)
print(result)

Seasonal_Sens_Slope_Test(slope=-5.0, intercept=73.29166666666667)


另一个例子

In [20]:
# -*- encoding: utf-8 -*-

import pymannkendall as mk
import numpy as np

# 生成3组数据
data1 = np.random.random(100)
data2 = np.arange(100)
data3 = np.arange(100,0,-1)
# print(data1)
# print(data2)
# print(data2)
for x in [data1,data2,data3]:
    result = mk.original_test(x,alpha=0.05)#alpha默认为0.05
    print(result)
    print(f'trend:{result.trend}','p_value:{:.2f}'.format(result.p),'slope:{:.2f}'.format(result.slope),sep = ',')



Mann_Kendall_Test(trend='no trend', h=False, p=0.614752145318523, z=0.5033017371129681, Tau=0.03434343434343434, s=170.0, var_s=112750.0, slope=0.0004604380232121828, intercept=0.4830796630609341)
trend:no trend,p_value:0.61,slope:0.00
Mann_Kendall_Test(trend='increasing', h=True, p=0.0, z=14.73869998208331, Tau=1.0, s=4950.0, var_s=112750.0, slope=1.0, intercept=0.0)
trend:increasing,p_value:0.00,slope:1.00
Mann_Kendall_Test(trend='decreasing', h=True, p=0.0, z=-14.73869998208331, Tau=-1.0, s=-4950.0, var_s=112750.0, slope=-1.0, intercept=100.0)
trend:decreasing,p_value:0.00,slope:-1.00


自定义的mk趋势检验函数

In [21]:
# -*- encoding: utf-8 -*-


# here put the import lib
from scipy.stats import norm
import numpy as np
from sklearn.linear_model import LinearRegression


def mk(x, alpha=0.1):
    n = len(x)

    # 计算趋势slope
    model = LinearRegression()
    model.fit(np.arange(1,n+1).reshape(-1,1),x)
    slope = model.coef_[0]

    # 计算S的值
    s = 0
    for j in range(n - 1):
        for i in range(j + 1, n):
            s += np.sign(x[i] - x[j])

    # 判断x里面是否存在重复的数，输出唯一数队列unique_x,重复数数量队列tp
    unique_x, tp = np.unique(x, return_counts=True)
    g = len(unique_x)

    # 计算方差VAR(S)
    if n == g:  # 如果不存在重复点
        var_s = (n * (n - 1) * (2 * n + 5)) / 18
    else:
        var_s = (n * (n - 1) * (2 * n + 5) - np.sum(tp * (tp - 1) * (2 * tp + 5))) / 18

    # 计算z_value
    if n <= 10:  # n<=10属于特例
        z = s / (n * (n - 1) / 2)
    else:
        if s > 0:
            z = (s - 1) / np.sqrt(var_s)
        elif s < 0:
            z = (s + 1) / np.sqrt(var_s)
        else:
            z = 0

    # 计算p_value，可以选择性先对p_value进行验证
    p = 2 * (1 - norm.cdf(abs(z)))

    # 计算Z(1-alpha/2)
    h = abs(z) > norm.ppf(1 - alpha / 2)

    # 趋势判断
    if (z < 0) and h:
        trend = -1#'decreasing'
    elif (z > 0) and h:
        trend = 1#'increasing'
    else:
        trend = 0#'no trend'

    return trend,p,slope


if __name__ == '__main__':
    data = np.random.random(100)
    data2 = np.arange(100)
    data3 = np.arange(100,0,-1)
    for x in [data,data2,data3]:
        trend,p,slope = mk(x,alpha = 0.05)
        print(f'trend:{trend}','p_value:{:.2f}'.format(p),'slope:{:.2f}'.format(slope),sep = ',')



trend:0,p_value:0.68,slope:-0.00
trend:1,p_value:0.00,slope:1.00
trend:-1,p_value:0.00,slope:-1.00
